# arXiv data extraction

This notebook provides code for extracting arXiv data following the dev documentation here: https://www.kaggle.com/datasets/Cornell-University/arxiv/code

#### 1. get arXiv data

In [1]:
# code snippet below taken from Kaggle docs

import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "arxiv-metadata-oai-snapshot.json"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "Cornell-University/arxiv",
  file_path,
  pandas_kwargs={"lines": True}
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

/Users/doriellelonke/Desktop/thesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/h9/9pwlclvs1wb_2zx0p9_rwn1m0000gn/T/ipykernel_51914/4196751292.py:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


#### 2. Extend anthropomorphic wordlists with WordNet

In [3]:
from tools import wordnet_syns as syns

extended_arg0_verbs = syns.extend_word_list('arg0_verbs','v')
extended_arg1_verbs = syns.extend_word_list('arg1_verbs','v')
extended_adjectives = syns.extend_word_list('adjectives','a')
extended_nouns = syns.extend_word_list('nouns','n')

#### 3. Get papers from arXiv in a iterable object and initiate keywords for matching relevant titles and abstracts

In [4]:
import re
import spacy
import pandas as pd
import pickle 

nlp = spacy.load("en_core_web_md")

paper_triplets = list(zip(df['id'],df['title'], df['abstract'].str.replace('\n', ' ')))
# zip 3-tuples of id, title and abstract

# for a case-insensitive re.match with words from title:
title_keywords = ['AI','LM','LLM','GPT','ChatGPT'] 

# for a case insensitive re.search in title:
title_phrases = ['artificial intelligence','language model']

# for a lemma-based string comparison against entities in the abstract:
keywords = ['AI','LM','LMs','LLM','LLMs','model','system','algorithm'] 
# spaCy lemmatizer does not handle plurals well for LM, LLM, so their plural version was included here too

#### 4. Functions for retrieving specific patterns for each class from the taxonomy of anthropomorphic structures

In [5]:
def arg0_active_criterion_check(sent,keywords,verb_list):

    check = 0

    for chunk in sent.noun_chunks:
        match = any(re.search(rf"\b{re.escape(word)}\b", chunk.text, re.IGNORECASE) for word in keywords)
        if match and chunk.root.dep_ == 'nsubj' and chunk.root.head.lemma_ in verb_list:
            check += 1

    if check > 0:
        return True
    else:
        return False

#### 5. General function for retreiving sentences matching a criterion 

In [7]:
def get_sentences(cat):

    with open(f"../preprocessed_data/arxiv_{cat}.txt","w") as file:

        done = False
        counter = 0 # initiate counter
        sentences_dict = {"SentenceID":[],"currentSentence":[],"prevSentence":[],"nextSentence":[],"Abstract":[]}
        stop_words = ['do','be','have','show'] 
        verb_list = [v for v in extended_arg0_verbs if v not in stop_words] # exclude stop words
    
        for idx,paper in enumerate(paper_triplets):

            paper_id = str(paper[0])
            title = str(paper[1])
            abstract = str(paper[2])

            if done:
                break

            title_words = [token.text for token in nlp(title)]
            keyword_match = any(re.match(keyword, word, re.IGNORECASE) for keyword in title_keywords for word in title_words)
            phrase_match = any(re.search(phrase, title.casefold(), re.IGNORECASE) for phrase in title_phrases)
        
            if keyword_match or phrase_match:
                doc = nlp(abstract)
            
                for i,sent in enumerate(doc.sents): # check for matches with the keywords in the noun chunks to find AI entities

                    if counter >= 1000:
                        done = True
                        break # stop when counter reaches 1000

                    sent_id = paper_id + "_" + str(idx) + "_" + str(i)

                    # check if at least one of the noun chunks is an AI entity whose root is an anthropomorphic predicate
                    if cat == "arg0_verbs_active":
                        criterion_met = arg0_active_criterion_check(sent,keywords,verb_list) 
                
                    if criterion_met:
                        counter += 1
                        file.write(sent_id+'\t'+sent.text+'\n')
                        sentences_dict["SentenceID"].append(sent_id)
                        sentences_dict["currentSentence"].append(list(doc.sents)[i].text)
                        sentences_dict["Abstract"].append(abstract)
                        try:
                            sentences_dict["prevSentence"].append(list(doc.sents)[i-1].text)
                        except IndexError:
                            sentences_dict["prevSentence"].append("")
                        try:
                            sentences_dict["nextSentence"].append(list(doc.sents)[i+1].text)
                        except IndexError:
                            sentences_dict["nextSentence"].append("")
                            
    return sentences_dict

#### Retrieve candidates for sentences in which the AI entity is the arg0 of an anthropomorphic predicate in the active voice. 

In [8]:
arg0_verbs_dict = get_sentences("arg0_verbs_active")
arg0_verbs_df = pd.DataFrame(data=arg0_verbs_dict)
arg0_verbs_df.to_pickle("../preprocessed_data/dataframes/arxiv_1000_arg0_verbs.pkl")